# Lab 2-2: MLflow Experiment Tracking

MLflow를 사용하여 ML 실험을 추적하고 관리하는 방법을 학습합니다.

## 학습 목표
- MLflow Tracking 기본 개념 이해
- 파라미터, 메트릭, 아티팩트 로깅
- 여러 실험 비교
- Model Registry 사용

## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install mlflow==2.9.2 scikit-learn pandas matplotlib -q
print("✅ 패키지 설치 완료!")

In [ ]:
# Import
import os
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print(f"MLflow Version: {mlflow.__version__}")

In [ ]:
# MLflow 서버 설정
# Kubeflow 환경에서 실행 시
MLFLOW_TRACKING_URI = os.getenv(
    'MLFLOW_TRACKING_URI',
    'http://mlflow-server-service.mlflow-system.svc.cluster.local:5000'
)

# 로컬에서 포트 포워딩 사용 시
# MLFLOW_TRACKING_URI = 'http://localhost:5000'

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print(f"MLflow Tracking URI: {MLFLOW_TRACKING_URI}")

# S3/MinIO 설정 (아티팩트 저장용)
os.environ['MLFLOW_S3_ENDPOINT_URL'] = os.getenv(
    'MLFLOW_S3_ENDPOINT_URL',
    'http://minio-service.kubeflow.svc:9000'
)
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID', 'minio')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY', 'minio123')

## 2. 데이터 준비

In [ ]:
# California Housing 데이터셋 로드
print("데이터 로드 중...")
data = fetch_california_housing()
X, y = data.data, data.target
feature_names = data.feature_names

# Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n데이터셋 정보:")
print(f"  - 전체 샘플: {len(X)}")
print(f"  - 학습 샘플: {len(X_train)}")
print(f"  - 테스트 샘플: {len(X_test)}")
print(f"  - 피처 수: {len(feature_names)}")
print(f"  - 피처: {list(feature_names)}")

## 3. MLflow 기본 사용법

### 3.1 실험(Experiment) 생성

In [ ]:
# 실험 설정
EXPERIMENT_NAME = "california-housing-lab"

# 실험 생성 또는 가져오기
experiment = mlflow.set_experiment(EXPERIMENT_NAME)

print(f"실험 정보:")
print(f"  - 이름: {experiment.name}")
print(f"  - ID: {experiment.experiment_id}")
print(f"  - 아티팩트 위치: {experiment.artifact_location}")

### 3.2 단일 Run 실행

In [ ]:
# 첫 번째 Run: RandomForest 기본 설정
with mlflow.start_run(run_name="rf-baseline"):
    
    # ===== 파라미터 로깅 =====
    params = {
        "n_estimators": 50,
        "max_depth": 5,
        "random_state": 42,
        "model_type": "RandomForestRegressor"
    }
    mlflow.log_params(params)
    print("✅ 파라미터 로깅 완료")
    
    # ===== 모델 학습 =====
    model = RandomForestRegressor(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        random_state=params["random_state"],
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    print("✅ 모델 학습 완료")
    
    # ===== 예측 및 평가 =====
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # ===== 메트릭 로깅 =====
    mlflow.log_metrics({
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "r2": r2
    })
    print("✅ 메트릭 로깅 완료")
    
    # ===== 태그 추가 =====
    mlflow.set_tag("stage", "development")
    mlflow.set_tag("author", "mlops-training")
    
    # ===== 모델 저장 =====
    mlflow.sklearn.log_model(model, "model")
    print("✅ 모델 저장 완료")
    
    # 결과 출력
    print(f"\n📊 모델 성능:")
    print(f"   - R2 Score: {r2:.4f}")
    print(f"   - RMSE: {rmse:.4f}")
    print(f"   - MAE: {mae:.4f}")

## 4. 여러 실험 실행 및 비교

In [ ]:
def run_experiment(model_class, model_params, run_name):
    """실험 실행 함수"""
    with mlflow.start_run(run_name=run_name):
        # 파라미터 로깅
        mlflow.log_params(model_params)
        mlflow.log_param("model_type", model_class.__name__)
        
        # 모델 학습
        model = model_class(**model_params)
        model.fit(X_train, y_train)
        
        # 평가
        y_pred = model.predict(X_test)
        
        metrics = {
            "mse": mean_squared_error(y_test, y_pred),
            "rmse": np.sqrt(mean_squared_error(y_test, y_pred)),
            "mae": mean_absolute_error(y_test, y_pred),
            "r2": r2_score(y_test, y_pred)
        }
        mlflow.log_metrics(metrics)
        
        # 모델 저장
        mlflow.sklearn.log_model(model, "model")
        
        print(f"[{run_name}] R2: {metrics['r2']:.4f}, RMSE: {metrics['rmse']:.4f}")
        
        return metrics

In [ ]:
# 여러 모델/하이퍼파라미터 실험
experiments = [
    # RandomForest 실험
    (RandomForestRegressor, {"n_estimators": 50, "max_depth": 5, "random_state": 42, "n_jobs": -1}, "rf-small"),
    (RandomForestRegressor, {"n_estimators": 100, "max_depth": 10, "random_state": 42, "n_jobs": -1}, "rf-medium"),
    (RandomForestRegressor, {"n_estimators": 200, "max_depth": 15, "random_state": 42, "n_jobs": -1}, "rf-large"),
    
    # GradientBoosting 실험
    (GradientBoostingRegressor, {"n_estimators": 100, "max_depth": 5, "random_state": 42}, "gb-baseline"),
    
    # Ridge 실험
    (Ridge, {"alpha": 1.0, "random_state": 42}, "ridge-baseline"),
]

print("=" * 60)
print("  여러 실험 실행")
print("=" * 60)

results = []
for model_class, params, name in experiments:
    metrics = run_experiment(model_class, params, name)
    results.append({"name": name, **metrics})

print("\n✅ 모든 실험 완료!")

### 4.1 실험 결과 비교

In [ ]:
# 결과를 DataFrame으로 정리
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('r2', ascending=False)

print("\n📊 실험 결과 비교 (R2 기준 정렬):")
print("=" * 60)
print(results_df.to_string(index=False))

In [ ]:
# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R2 Score 비교
colors = plt.cm.viridis(np.linspace(0, 1, len(results_df)))
axes[0].barh(results_df['name'], results_df['r2'], color=colors)
axes[0].set_xlabel('R2 Score')
axes[0].set_title('R2 Score by Model')
axes[0].axvline(x=results_df['r2'].max(), color='red', linestyle='--', label='Best')

# RMSE 비교
axes[1].barh(results_df['name'], results_df['rmse'], color=colors)
axes[1].set_xlabel('RMSE')
axes[1].set_title('RMSE by Model')
axes[1].axvline(x=results_df['rmse'].min(), color='red', linestyle='--', label='Best')

plt.tight_layout()
plt.savefig('experiment_comparison.png', dpi=100, bbox_inches='tight')
plt.show()

print("\n📈 그래프 저장: experiment_comparison.png")

## 5. MLflow UI에서 결과 확인

MLflow UI에서 실험 결과를 확인하세요:

1. 브라우저에서 `http://localhost:5000` 접속
2. 왼쪽 사이드바에서 `california-housing-lab` 실험 선택
3. 각 Run을 클릭하여 상세 정보 확인
4. 여러 Run 선택 후 **Compare** 버튼 클릭

## 6. Model Registry 사용

In [ ]:
# 최고 성능 모델 등록
best_model_name = results_df.iloc[0]['name']
print(f"최고 성능 모델: {best_model_name}")

# 해당 Run 찾기
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    filter_string=f"tags.mlflow.runName = '{best_model_name}'"
)

if len(runs) > 0:
    best_run_id = runs.iloc[0]['run_id']
    print(f"Run ID: {best_run_id}")
    
    # 모델 등록
    model_uri = f"runs:/{best_run_id}/model"
    registered_model = mlflow.register_model(
        model_uri=model_uri,
        name="california-housing-model"
    )
    
    print(f"\n✅ 모델 등록 완료!")
    print(f"   - 이름: {registered_model.name}")
    print(f"   - 버전: {registered_model.version}")

## 7. 등록된 모델 로드 및 사용

In [ ]:
# 모델 로드
model_name = "california-housing-model"
model_version = 1  # 또는 "latest"

try:
    loaded_model = mlflow.sklearn.load_model(
        model_uri=f"models:/{model_name}/{model_version}"
    )
    print(f"✅ 모델 로드 완료: {model_name} v{model_version}")
    
    # 테스트 예측
    test_sample = X_test[:5]
    predictions = loaded_model.predict(test_sample)
    
    print(f"\n테스트 예측:")
    for i, (pred, actual) in enumerate(zip(predictions, y_test[:5])):
        print(f"  Sample {i+1}: Predicted={pred:.2f}, Actual={actual:.2f}")
        
except Exception as e:
    print(f"⚠️ 모델 로드 실패: {e}")
    print("   Model Registry에 모델이 등록되어 있는지 확인하세요.")

## ✅ Lab 2-2 완료!

### 학습한 내용
- `mlflow.set_experiment()`: 실험 생성/선택
- `mlflow.start_run()`: Run 시작
- `mlflow.log_params()`: 파라미터 로깅
- `mlflow.log_metrics()`: 메트릭 로깅
- `mlflow.sklearn.log_model()`: 모델 저장
- `mlflow.register_model()`: 모델 등록
- `mlflow.sklearn.load_model()`: 모델 로드

### 다음 단계
- Lab 2-3: KServe 배포